In [ ]:
!pip install datasets
!pip install evaluate
!pip install bert_score
!pip install bitsandbytes
!pip install peft
!pip install textstat
# !pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import re, time, random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn.functional as F
import transformers
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    T5ForConditionalGeneration,
    T5Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import (
    PeftModel,
    PeftConfig,
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType
)
import evaluate
from bert_score import score as bert_score
import textstat
from datasets import load_dataset, Dataset
from tqdm import tqdm
import bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

#Load in LLaMA
def load_llama_model():
    model_name = "meta-llama/Llama-3.2-3B-Instruct"
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    return model, tokenizer

#Preprocess the Reveal Dataset
def preprocess_datasets():
    openbookqa = load_dataset("openbookqa", "main")
    reveal = load_dataset("google/reveal")

    def filter_bad_chains(example):
        if 'explanation' in example and example['explanation']:
            return len(example['explanation'].split()) > 5
        return False

    filtered_openbookqa = openbookqa.filter(filter_bad_chains)
    return filtered_openbookqa, reveal




In [ ]:
# !huggingface-cli login


In [ ]:
llama_model, llama_tokenizer = load_llama_model()


config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
dataset_openbookqa, dataset_reveal = preprocess_datasets()


README.md:   0%|          | 0.00/9.06k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/496k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/58.2k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4957 [00:00<?, ? examples/s]

...

Generating eval split:   0%|          | 0/4956 [00:00<?, ? examples/s]

Generating open split:   0%|          | 0/1146 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4957 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

Filter:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:

#Verifier Finetuned in "Finetune_Gemma2b.ipynb" notebook, loading it in here
def load_finetuned_verifier():
    peft_model_path = "/content/drive/MyDrive/Colab Notebooks/DS266/Final_Project/gemma_verifier_enhanced_lora_final_100"

    # Load PEFT configuration + Base Verifier Model (FP16)
    peft_config = PeftConfig.from_pretrained(peft_model_path)
    base_model = AutoModelForCausalLM.from_pretrained(
        peft_config.base_model_name_or_path,
        torch_dtype=torch.float16,
        device_map="auto"
    )

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(peft_model_path)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        base_model.config.pad_token_id = tokenizer.pad_token_id

    # Load fine-tuned LoRA layers onto the base model
    model = PeftModel.from_pretrained(base_model, peft_model_path)

    # Prediction Function with Few shot examples
    def predict_validity(question, step, previous_steps=None, step_type="Unknown"):

        is_logical_step = "Logical" in step_type

        # Format previous steps (if they exist)
        previous_steps_text = ""
        if previous_steps and len(previous_steps) > 0:
            previous_steps_text = "Previous steps:\n" + "\n".join(f"- {s}" for s in previous_steps if s) + "\n\n"

        # Prompt Engineering for Verifier (including Few-Shot Examples)
        if is_logical_step:
            instruction = (
                "You are a reasoning verifier. Your task is to determine if a reasoning step is logically correct "
                "given the context of a question and previous reasoning steps. Focus on whether the step follows "
                "logically from what came before, not just whether it's factually accurate on its own."
            )
            few_shot_example = (
                "Example 1:\n"
                "Question: What is 5 + 7?\n"
                "Previous steps:\n"
                "- 5 + 7 can be calculated directly.\n"
                "Reasoning Step: The sum of 5 and 7 is 12.\n"
                "Is the reasoning step valid? Yes, this reasoning step is valid.\n\n"
                "Example 2:\n"
                "Question: How many planets are in our solar system?\n"
                "Previous steps:\n"
                "- There are 8 recognized planets in our solar system.\n"
                "- Pluto used to be considered the 9th planet.\n"
                "Reasoning Step: Therefore, there are 9 planets in our solar system.\n"
                "Is the reasoning step valid? No, this reasoning step is invalid. The previous step states that Pluto is no longer considered a planet.\n\n"
            )
        else:
            instruction = (
                "You are a reasoning verifier. Your task is to determine if a reasoning step is valid "
                "given the context of a question. Evaluate whether the step is reasonable and contributes "
                "to answering the question."
            )
            few_shot_example = (
                "Example 1:\n"
                "Question: What is the capital of France?\n"
                "Reasoning Step: Paris is the capital of France.\n"
                "Is the reasoning step valid? Yes, this reasoning step is valid.\n\n"
                "Example 2:\n"
                "Question: What is 2+2?\n"
                "Reasoning Step: 2+2=5\n"
                "Is the reasoning step valid? No, this reasoning step is invalid.\n\n"
            )

        # Format task
        task = (
            f"Question: {question}\n"
            f"{previous_steps_text}"
            f"Reasoning Step: {step}\n"
            f"Is the reasoning step valid?"
        )

        # Combine everything into full prompt
        input_text = f"{instruction}\n\n{few_shot_example}{task}"
        inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.1,
                do_sample=False,
                top_p=0.95,
            )

        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # Extract the prediction
        response = prediction.split("Is the reasoning step valid?")[-1].strip()
        contains_explanation = len(response.split()) > 5

        #Case 1:
        # Determine if valid or invalid. This is a manual approach of verification indication.
        valid_indicators = ["yes", "valid", "correct", "accurate", "true"]
        invalid_indicators = ["no", "invalid", "incorrect", "wrong", "false", "error", "mistake"]
        has_valid = any(indicator in response.lower() for indicator in valid_indicators)
        has_invalid = any(indicator in response.lower() for indicator in invalid_indicators)

        #Case 2:
        # Look for specifc phrases, again manual approach to verification indication.
        if (has_valid and has_invalid) or (not has_valid and not has_invalid):
            if "yes, this reasoning step is valid" in response.lower():
                return True
            elif "no, this reasoning step is invalid" in response.lower():
                return False


            #Case 3:
            # If there's an explanation, do more nuanced analysis
            if contains_explanation:

                # Count positive and negative indicators to determine final output
                valid_count = sum(response.lower().count(indicator) for indicator in valid_indicators)
                invalid_count = sum(response.lower().count(indicator) for indicator in invalid_indicators)
                if valid_count > invalid_count:
                    return True
                elif invalid_count > valid_count:
                    return False

            #Case 4
            #If "yes" or "no" is at the start
            if response.lower().startswith("yes"):
                return True
            elif response.lower().startswith("no"):
                return False

        #Case 5:
        # If only valid indicators
        if has_valid and not has_invalid:
            return True

        #Case 6:
        # If only invalid indicators
        if has_invalid and not has_valid:
            return False

        #Case 7:
        # Stay conservative and return this as False (better to have a False Negative than False Positive)
        return False

    return model, tokenizer, predict_validity


In [ ]:

def test_model(verifier_model, verifier_tokenizer, predict_validity):
    test_cases = [
        # Fact cases (simple validity)
        {"question": "What is the capital of France?",
         "step": "Paris is the capital of France.",
         "expected": True,
         "step_type": "Attribution step."},
        {"question": "What is 2+2?",
         "step": "2+2 equals 4.",
         "expected": True,
         "step_type": "Attribution step."},
        {"question": "What is the capital of France?",
         "step": "London is the capital of France.",
         "expected": False,
         "step_type": "Attribution step."},
        {"question": "What is the capital of France?",
         "step": "Hot dog.",
         "expected": False,
         "step_type": "Attribution step."},

        # Logic reasoning cases
        {"question": "How many planets are in our solar system?",
         "step": "Therefore, there are 8 planets in our solar system.",
         "previous_steps": ["There are 8 recognized planets in our solar system.",
                           "Pluto was reclassified as a dwarf planet."],
         "expected": True,
         "step_type": "Logical step."},
        {"question": "How many planets are in our solar system?",
         "step": "Therefore, there are 9 planets in our solar system.",
         "previous_steps": ["There are 8 recognized planets in our solar system.",
                           "Pluto was reclassified as a dwarf planet."],
         "expected": False,
         "step_type": "Logical step."},

        # Cases from the REVEAL dataset
        {"question": "If microchip makers didn't keep trying to shrink their chips, how fast could your average home computer be by now?",
         "step": "The speed of a computer is determined by the number of transistors on the chip.",
         "expected": True,
         "step_type": "Attribution step."},
        {"question": "If microchip makers didn't keep trying to shrink their chips, how fast could your average home computer be by now?",
         "step": "The number of transistors on a chip is directly proportional to the size of the chip.",
         "expected": True,
         "step_type": "Attribution step."},
        {"question": "If microchip makers didn't keep trying to shrink their chips, how fast could your average home computer be by now?",
         "step": "Thus, if microchip makers didn't keep trying to shrink their chips, the speed of the average home computer would not have increased significantly.",
         "previous_steps": ["The speed of a computer is determined by the number of transistors on the chip.",
                           "The number of transistors on a chip is directly proportional to the size of the chip."],
         "expected": False,
         "step_type": "Logical step."},
    ]

    results = []
    for i, test in enumerate(test_cases):
        # Extract test case
        question = test["question"]
        step = test["step"]
        previous_steps = test.get("previous_steps", None)
        step_type = test.get("step_type", "Unknown")
        expected = test["expected"]

        # Get prediction
        is_valid = predict_validity(question, step, previous_steps, step_type)
        correct = is_valid == expected

        # Store results
        results.append({
            "case": i+1,
            "question": question,
            "step": step,
            "previous_steps": previous_steps,
            "step_type": step_type,
            "predicted": "valid" if is_valid else "invalid",
            "expected": "valid" if expected else "invalid",
            "correct": correct
        })

    # Calculate accuracy
    accuracy = sum(1 for r in results if r["correct"]) / len(results)
    print("Accuracy on test cases", accuracy)

    # Calculate accuracy by step type
    attribution_results = [r for r in results if "Attribution" in r["step_type"]]
    logical_results = [r for r in results if "Logical" in r["step_type"]]

    if attribution_results:
        attribution_accuracy = sum(1 for r in attribution_results if r["correct"]) / len(attribution_results)
        print("Accuracy on Attribution steps:", attribution_accuracy)
    else:
        print("No Attribution steps found")

In [ ]:


verifier_model, verifier_tokenizer, predict_validity = load_finetuned_verifier()


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Extract individual reasoning steps from the chain with improved filtering
def extract_reasoning_steps(reasoning_chain):

    # Skip question prefix if present
    if "Q:" in reasoning_chain and "Let's reason step by step" in reasoning_chain:
        content = reasoning_chain.split("Let's reason step by step")[-1].strip()
    else:
        content = reasoning_chain

    # Check for step numbering (Step 1, Step 2) or numbered lists (1., 2.)
    if "Step" in content or any(re.search(r"\b\d+[\.\:]", line) for line in content.split("\n")):
        lines = content.split("\n")
        steps, current_step, in_step = [], "", False

        for line in lines:
            line = line.strip()
            if re.search(r"\bStep\s+\d+[\.\:]", line) or re.search(r"^\d+[\.\:]", line):
                if current_step and in_step:
                    steps.append(current_step.strip())
                current_step, in_step = line, True
            elif in_step and line:
                current_step += " " + line

        if current_step and in_step:
            steps.append(current_step.strip())

        if steps:
            return steps

    # Check for markdown headers (## Sectioned Steps)
    if "##" in content:
        sections = content.split("##")
        steps = [section.strip() for section in sections[1:] if section.strip()]
        if steps:
            return steps

    # Default to sentence splitting
    sentences = re.split(r'(?<=[.!?])\s+', content)
    return [s.strip() for s in sentences if s.strip() and len(s) > 10]


#Compute the average pairwise cosine similarity between reasoning steps.
def coherence_score(steps):
    if len(steps) < 2:
        return 1.0  # Fully coherent if single step

    vectorizer = TfidfVectorizer().fit_transform(steps)
    similarity_matrix = cosine_similarity(vectorizer, vectorizer)
    return similarity_matrix.mean()  # Average pairwise similarity


#Runs iterative refinement up to 3 times, fixing invalid steps and tracking performance.
def iterative_refinement(generator_model, generator_tokenizer, verifier_model, verifier_tokenizer,
                         question, predict_validity):
    input_prompt = f"Question: {question}\n Reason step by step."
    best_reasoning, refinement_attempts = None, 0
    start_time = time.time()  # Start tracking refinement time

    for iteration in range(3):  # Max 3 refinements
        refinement_attempts += 1

        # Generate reasoning
        inputs = generator_tokenizer(input_prompt, return_tensors="pt").to("cuda")
        outputs = generator_model.generate(**inputs, max_length=200)
        reasoning_chain = generator_tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Iteration {iteration + 1}: Generated Reasoning:\n{reasoning_chain}\n{'-'*50}")


        # Extract individual steps
        steps = extract_reasoning_steps(reasoning_chain)

        # Verify each step
        invalid_steps = []
        print(f"Extracted Steps for Iteration {iteration + 1}:")
        for i, step in enumerate(steps):
            print(f"Step {i+1}: {step}")
            previous_steps = steps[:i] if i > 0 else None
            step_type = "Attribution step." if i == 0 else "Logical step."
            is_valid = predict_validity(question, step, previous_steps, step_type)

            if not is_valid:
                invalid_steps.append((i, step))
        print('-' * 50)

        # If all steps are valid, return reasoning
        if not invalid_steps:
            refinement_time = time.time() - start_time
            return reasoning_chain, refinement_attempts, refinement_time

        # Store best reasoning attempt (if less than half steps are invalid)
        if best_reasoning is None and len(invalid_steps) <= len(steps) / 2:
            best_reasoning = reasoning_chain

        # Initialize feedback before appending to it
        feedback = ""
        for idx, step in invalid_steps:
            feedback += f"- Step {idx+1} is invalid: \"{step}\"\n"
        feedback += (
    "For each invalid step, rewrite it to ensure it follows from the previous steps. Identify incorrect assumptions and provide a revised version. "
    "Break down each step logically, verify assumptions, and ensure consistency in your thought process. "
    "If any gaps, ambiguities, or contradictions exist, correct them by providing clearer justifications. ")

        input_prompt = f"Q: {question}\n{feedback}\n"



    refinement_time = time.time() - start_time
    return best_reasoning if best_reasoning else reasoning_chain, refinement_attempts, refinement_time







def run_experiment(generator_model, generator_tokenizer, verifier_model, verifier_tokenizer,
                   dataset, num_samples=100, predict_validity=None):

    results = []

    # Convert HF Dataset to list of dictionaries
    if isinstance(dataset, Dataset):
        dataset = dataset.to_pandas().to_dict(orient="records")  # Convert Dataset -> Pandas -> List of Dicts
    if isinstance(dataset, dict):
        dataset = list(dataset.values())
    elif isinstance(dataset, pd.DataFrame):
        dataset = dataset.to_dict(orient="records")

    dataset_sample = random.sample(dataset, min(num_samples, len(dataset))) if isinstance(dataset, list) else dataset
    for example in tqdm(dataset_sample):
        question = example.get("question", "")
        if not question:
            continue

        try:
            # Generate initial reasoning
            inputs = generator_tokenizer(f"Q: {question}\nLet's reason step by step.", return_tensors="pt").to("cuda")
            outputs = generator_model.generate(**inputs, max_length=200)
            initial_reasoning = generator_tokenizer.decode(outputs[0], skip_special_tokens=True)

            initial_steps = extract_reasoning_steps(initial_reasoning)
            initial_invalid_steps = [
                (i, step) for i, step in enumerate(initial_steps)
                if not predict_validity(question, step, initial_steps[:i] if i > 0 else None, "Logical step.")
            ]

            # Run iterative refinement
            refined_reasoning, refinement_attempts, refinement_time = iterative_refinement(
                generator_model, generator_tokenizer,
                verifier_model, verifier_tokenizer,
                question, predict_validity
            )

            # Verify refined steps
            final_steps = extract_reasoning_steps(refined_reasoning)
            final_invalid_steps = [
                (i, step) for i, step in enumerate(final_steps)
                if not predict_validity(question, step, final_steps[:i] if i > 0 else None, "Logical step.")
            ]

            # Compute metrics
            result = {
                "question": question,
                "initial_reasoning": initial_reasoning,
                "final_reasoning": refined_reasoning,
                "initial_step_count": len(initial_steps),
                "final_step_count": len(final_steps),
                "initial_word_count": len(initial_reasoning.split()),
                "final_word_count": len(refined_reasoning.split()),
                "initial_invalid_count": len(initial_invalid_steps),
                "final_invalid_count": len(final_invalid_steps),
                "reached_valid_reasoning": len(final_invalid_steps) == 0,
                "step_count_change": len(final_steps) - len(initial_steps),
                "word_count_change": len(refined_reasoning.split()) - len(initial_reasoning.split()),
                "initial_readability": textstat.flesch_kincaid_grade(initial_reasoning),
                "final_readability": textstat.flesch_kincaid_grade(refined_reasoning),
                "readability_change": textstat.flesch_kincaid_grade(refined_reasoning) - textstat.flesch_kincaid_grade(initial_reasoning),
                "initial_coherence": coherence_score(initial_steps),
                "final_coherence": coherence_score(final_steps),
                "coherence_change": coherence_score(final_steps) - coherence_score(initial_steps),
                "refinement_attempts": refinement_attempts,
                "refinement_time_seconds": refinement_time
            }
            results.append(result)

        except Exception as e:
            print(f"Error processing example: {e}")

    return pd.DataFrame(results)


In [ ]:
#Load Dataset
dataset_gsm8k = load_dataset("gsm8k", "main")["train"]
dataset_gsm8k = [
    {"question": ex["question"], "fact1": ex["answer"]}
    for ex in dataset_gsm8k
]


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [ ]:

# Run experiment
results_df_gsm8k = run_experiment(
    llama_model, llama_tokenizer,
    verifier_model, verifier_tokenizer,
    dataset_gsm8k, num_samples=100,
    predict_validity=predict_validity
)



  0%|          | 0/100 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Iteration 1: Generated Reasoning:
Question: Elysse can carry 3 bags of groceries into her home with each trip from the car. Her brother can carry the same amount. How many trips will it take them to carry 30 bags of groceries?
 Reason step by step. 
Let's break down the problem step by step.

## Step 1: Determine the number of bags each person can carry per trip.
Elysse and her brother can each carry 3 bags of groceries per trip.

## Step 2: Calculate the total number of bags they can carry per trip.
Together, they can carry 3 + 3 = 6 bags of groceries per trip.

## Step 3: Determine how many trips it will take to carry 30 bags of groceries.
To find the number of trips, we divide the total number of bags (30) by the number of bags they can carry per trip (6). trips = 30 / 6 = 5.

The final answer is: $\boxed
--------------------------------------------------
Extracted Steps for Iteration 1:
Step 1: ## Step 1: Determine the number of bags each person can carry per trip. Elysse and her b

  1%|          | 1/100 [00:58<1:37:17, 58.97s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Iteration 1: Generated Reasoning:
Question: Joe played catch with Derek and Tammy. He caught the ball 23 times. Derek made four less than double the catches Joe did. Tammy caught the ball sixteen more than a third of the times Derek did. How many times did Tammy catch the ball?
 Reason step by step. Step 1:  Derek made four less than double the catches Joe did. Joe caught the ball 23 times, so double that is 2 x 23 = 46. Four less than that is 46 - 4 = 42.
 Step 2:  Tammy caught the ball sixteen more than a third of the times Derek did. Derek caught the ball 42 times, so a third of that is 42 / 3 = 14. Sixteen more than that is 14 + 16 = 30.

The final answer is: $\boxed{30}$
--------------------------------------------------
Extracted Steps for Iteration 1:
Step 1: Reason step by step. Step 1:  Derek made four less than double the catches Joe did. Joe caught the ball 23 times, so double that is 2 x 23 = 46. Four less than that is 46 - 4 = 42.
Step 2: Step 2:  Tammy caught the ball six

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

 98%|█████████▊| 98/100 [2:07:25<02:43, 81.73s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Iteration 1: Generated Reasoning:
Question: James and John combine their friends lists.  James has 75 friends.  John has 3 times as many friends as James.  They share 25 friends.  How many people are on the combined list?
 Reason step by step. To solve this problem, we can follow these steps:
Step 1: Calculate the number of friends John has.
Since John has 3 times as many friends as James, and James has 75 friends, we can multiply the number of friends James has by 3 to find the number of friends John has.
John's friends = 3 x 75 = 225
Step 2: Add the number of friends James and John have together.
Combined number of friends = James' friends + John's friends = 75 + 225 = 300
Step 3: Subtract the number of friends that James and John share from the combined list.
Since James and John share 25 friends, we need to subtract 25 from
--------------------------------------------------
Extracted Steps for Iteration 1:
Step 1: Step 1: Calculate the number of friends John has. Since John has 3 t

 99%|█████████▉| 99/100 [2:09:10<01:28, 88.83s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Iteration 1: Generated Reasoning:
Question: The class has to guess how many jellybeans are in a jar the teacher brought in. He lets 4 students make their guesses. The first thinks it contains 100 jellybeans. The second says there are 8 times as many. The third guesses 200 less than the third. The fourth takes the average of the first three guesses, and then adds 25 to it. How many jellybeans did the fourth student guess?
 Reason step by step. The first student guessed 100 jellybeans.
 The second student guessed 8 times the number of the first student. So the second student guessed 8 * 100 = 800 jellybeans.
 The third student guessed 200 less than the second student. So the third student guessed 800 - 200 = 600 jellybeans.
 To find the average of the first three guesses, we need to add the three numbers and divide by 3. So the average is (100 + 800 + 600) / 
--------------------------------------------------
Extracted Steps for Iteration 1:
Step 1: Question: The class has to guess how m

100%|██████████| 100/100 [2:12:34<00:00, 79.54s/it] 


In [ ]:
display(results_df_gsm8k)

,question,initial_reasoning,final_reasoning,initial_step_count,final_step_count,initial_word_count,final_word_count,initial_invalid_count,final_invalid_count,reached_valid_reasoning,step_count_change,word_count_change,initial_readability,final_readability,readability_change,initial_coherence,final_coherence,coherence_change,refinement_attempts,refinement_time_seconds
0,Elysse can carry 3 bags of groceries into her ...,Q: Elysse can carry 3 bags of groceries into h...,Question: Elysse can carry 3 bags of groceries...,2,3,137,157,0,0,True,1,20,4.8,4.2,-0.6,0.706596,0.644456,-0.062140,1,23.073514
1,Joe played catch with Derek and Tammy. He caug...,Q: Joe played catch with Derek and Tammy. He c...,Q: Joe played catch with Derek and Tammy. He c...,2,2,146,153,0,0,True,0,7,2.8,5.1,2.3,0.777793,0.667282,-0.110511,2,28.718240
2,James has 20 pairs of red socks and half as ma...,Q: James has 20 pairs of red socks and half as...,Q: James has 20 pairs of red socks and half as...,4,2,156,157,0,1,False,-2,1,3.3,7.1,3.8,0.715255,0.806162,0.090908,3,98.800337
3,Boston had .5 feet of snow on the first day of...,Q: Boston had.5 feet of snow on the first day ...,Q: Boston had.5 feet of snow on the first day ...,2,11,144,157,0,0,True,9,13,2.8,6.4,3.6,0.896187,0.179852,-0.716335,2,136.159256
4,There was only enough room for 21 people on a ...,Q: There was only enough room for 21 people on...,Question: There was only enough room for 21 pe...,5,8,160,129,0,0,True,3,-31,6.3,5.8,-0.5,0.338870,0.255421,-0.083449,1,51.045488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Krista started raising chickens. She has 10 he...,Q: Krista started raising chickens. She has 10...,Question: Krista started raising chickens. She...,2,3,146,153,0,0,True,1,7,3.6,7.0,3.4,0.654476,0.668791,0.014315,1,22.251302
88,"Shannon bought 5 pints of frozen yogurt, two p...","Q: Shannon bought 5 pints of frozen yogurt, tw...",Question: Shannon bought 5 pints of frozen yog...,3,8,154,152,1,0,True,5,-2,8.6,5.6,-3.0,0.590529,0.247405,-0.343124,1,51.678905
89,Clare's mother gave her $47 to go to the store...,Q: Clare's mother gave her $47 to go to the st...,Question: Clare's mother gave her $47 to go to...,3,4,149,126,1,0,True,1,-23,3.5,1.8,-1.7,0.827442,0.509593,-0.317848,1,27.894570
90,James and John combine their friends lists. J...,Q: James and John combine their friends lists....,Question: James and John combine their friends...,10,3,143,154,3,0,True,-7,11,1.9,3.8,1.9,0.231852,0.630252,0.398401,1,22.863183


In [ ]:
results_df_gsm8k.to_csv("gsm8k_data_evaluated.csv", index=False)
